In [1]:
import requests
import pandas as pd
import json
import re
from tqdm import tqdm

tqdm.pandas()

In [2]:
data = pd.read_csv("../database/review_1.csv", encoding="utf-8")
data[:5]

,_id,cleanliness,content,convenience,date,hotelAddr,hotelGrade,hotelLoc,hotelName,kindness,position,reviewID,room,totalScore,user
0,609182b3f5c0adc042e03afd,5,영등포에서는 이곳만 와요 너무 좋습니다,5,2021-05-04,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12759201,[특가야놀자☆선착순 고층 룸UP+25시간] Premier King,5,지금사랑해0028
1,609182b3f5c0adc042e03b08,5,체크인 시간이 너무너무 깁니다 빨리 할 수 있는 방법을 구상해야 할 것 같습니다,5,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,1,5,12752310,[특가야놀자☆선착순 고층 룸UP+27시간] Premier King,3,엄마구출
2,609182b3f5c0adc042e03b0a,5,잘머물다갑니다,5,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12752105,[특가야놀자☆선착순 고층 룸UP+27시간스테이] Premier Twin,5,그때그시절처렁
3,609182b3f5c0adc042e03b0b,5,최근에 침대가 너무 삐걱거림을 느끼고 있네요 ㅜㅜ,2,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12751866,[특가야놀자☆선착순 고층 룸UP+27시간스테이] Premier Twin,4,동별
4,609182b3f5c0adc042e03b13,5,잘 쉬다 가요!!!!!!!!!!!!!!!!!!!!!!!!! !!!!!!!!!,4,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,2,5,12747672,[특가야놀자☆10일간 Spring Flash Sale] Premier King,3,폭신폭신꿀보0006


In [3]:
base_url = "https://m.search.naver.com/p/csearch/ocontent/util/SpellerProxy"
header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
parameter = {
    "_callback": "jQuery112408684363825005834_1636349956720",
    "where": "nexearch",
    "q": "조았어요",
    "color_blindness": "0",
    # "_": "1620612671606"
}

In [4]:
req = requests.get(base_url, params=parameter, headers=header)

In [5]:
items = re.findall("\(([^)]+)", req.text)[0]

In [6]:
ex = json.loads(items)["message"]["result"]["html"]
ex

"<em class='red_text'>좋았어요</em>"

In [7]:
def remove_tag(content):
   # remove tags in text string
   cleaner = re.compile("<.*?>")
   cleaned_text = re.sub(cleaner, "", content)

   return cleaned_text

In [8]:
sample = remove_tag(ex)
print(sample)

좋았어요


In [9]:
def check_text(x):
    base_url = "https://m.search.naver.com/p/csearch/ocontent/util/SpellerProxy"
    header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    parameter = {
        "_callback": "jQuery1124013465754649774486_1620612671605",
        "where": "nexearch",
        "q": str(x),
        "color_blindness": "0",
        # "_": "1620612671606"
    }

    req = requests.get(base_url, params=parameter, headers=header)

    try:
        items = re.findall("\(([^)]+)", req.text)[0]
        checked = json.loads(items)["message"]["result"]["html"]
        tag_removed = remove_tag(checked)

        return tag_removed

    except:
        return x

In [10]:
data["content"] = data["content"].progress_apply(lambda x: check_text(x))
data[0:5]

100%|██████████| 99000/99000 [1:53:44<00:00, 14.51it/s]


,_id,cleanliness,content,convenience,date,hotelAddr,hotelGrade,hotelLoc,hotelName,kindness,position,reviewID,room,totalScore,user
0,609182b3f5c0adc042e03afd,5,영등포에서는 이곳만 와요 너무 좋습니다,5,2021-05-04,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12759201,[특가야놀자☆선착순 고층 룸UP+25시간] Premier King,5,지금사랑해0028
1,609182b3f5c0adc042e03b08,5,체크인 시간이 너무너무 깁니다 빨리할 수 있는 방법을 구상해야 할 것 같습니다,5,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,1,5,12752310,[특가야놀자☆선착순 고층 룸UP+27시간] Premier King,3,엄마구출
2,609182b3f5c0adc042e03b0a,5,잘 머물다 갑니다,5,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12752105,[특가야놀자☆선착순 고층 룸UP+27시간스테이] Premier Twin,5,그때그시절처렁
3,609182b3f5c0adc042e03b0b,5,최근에 침대가 너무 삐걱거림을 느끼고 있네요 ㅜㅜ,2,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12751866,[특가야놀자☆선착순 고층 룸UP+27시간스테이] Premier Twin,4,동별
4,609182b3f5c0adc042e03b13,5,잘 쉬다 가요!!!!!!!!!!!!!!!!!!!!!!!!! !!!!!!!!!,4,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,2,5,12747672,[특가야놀자☆10일간 Spring Flash Sale] Premier King,3,폭신폭신꿀보0006


In [36]:
data.to_csv("../database/review_2_naver.csv", encoding="utf-8", index=False)